# Iran BGP Traffic Analysis: What Actually Happened

Analyzing real BGP routing data during Iran's internet shutdowns.

In [ ]:
import requests
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import numpy as np

# Key Iranian ASNs
IRAN_ASNS = [58224, 12880, 197207, 44244, 31549, 42337, 43754, 49666, 57218]

print("BGP Shutdown Analysis")
print("="*60)

In [ ]:
# Get BGP routing status over time for Iran (country-level)
print("Fetching Iran BGP visibility data...")

# RIPEstat BGP State endpoint - visibility of Iran's prefixes
url = "https://stat.ripe.net/data/country-routing-stats/data.json?resource=IR"
resp = requests.get(url, timeout=60)
country_routing = resp.json().get('data', {}) if resp.status_code == 200 else {}

if country_routing:
    print(f"\nIran Routing Statistics:")
    stats = country_routing.get('stats', {})
    for key, value in stats.items():
        print(f"  {key}: {value}")

In [ ]:
# Get historical BGP updates for key Iranian ASN during shutdown periods
# November 2019 shutdown: Nov 16-23, 2019
# September 2022 shutdown: Sep 21 onwards

def get_bgp_updates(asn, starttime, endtime):
    """Get BGP update activity for an ASN during a time period"""
    url = f"https://stat.ripe.net/data/bgp-updates/data.json?resource=AS{asn}&starttime={starttime}&endtime={endtime}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

def get_bgp_state(resource, timestamp):
    """Get BGP routing state at a specific time"""
    url = f"https://stat.ripe.net/data/bgp-state/data.json?resource={resource}&timestamp={timestamp}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

print("API functions defined")

In [ ]:
# Analyze November 2019 shutdown
print("\n" + "="*60)
print("NOVEMBER 2019 SHUTDOWN ANALYSIS")
print("="*60)

# Get BGP updates for TIC (AS58224) around Nov 2019 shutdown
# Shutdown started ~Nov 16, 2019
print("\nFetching BGP updates for AS58224 (TIC) - Nov 2019...")

updates_nov2019 = get_bgp_updates(58224, "2019-11-14T00:00", "2019-11-25T00:00")

if updates_nov2019:
    updates = updates_nov2019.get('updates', [])
    print(f"Total BGP updates in period: {len(updates)}")
    
    # Count by type
    announcements = [u for u in updates if u.get('type') == 'A']
    withdrawals = [u for u in updates if u.get('type') == 'W']
    
    print(f"  Announcements: {len(announcements)}")
    print(f"  Withdrawals: {len(withdrawals)}")

In [ ]:
# Get visibility/reachability data over time
print("\nFetching BGP visibility timeline...")

def get_visibility(resource, starttime, endtime):
    """Get visibility/routing status over time"""
    url = f"https://stat.ripe.net/data/visibility/data.json?resource={resource}&starttime={starttime}&endtime={endtime}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

# Check visibility of major Iranian prefix during shutdown
vis_data = get_visibility("AS58224", "2019-11-14", "2019-11-25")
if vis_data:
    visibilities = vis_data.get('visibilities', [])
    print(f"Visibility data points: {len(visibilities)}")

In [ ]:
# Get RRC (Route Collector) data showing prefix visibility
print("\nFetching RIS routing data...")

def get_ris_prefixes(asn, time_point):
    """Get prefixes seen from RIS route collectors at a specific time"""
    url = f"https://stat.ripe.net/data/ris-prefixes/data.json?resource=AS{asn}&query_time={time_point}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

# Compare prefix counts: before, during, and after Nov 2019 shutdown
time_points = [
    ("2019-11-14T12:00", "Before shutdown"),
    ("2019-11-17T12:00", "During shutdown (Day 2)"),
    ("2019-11-20T12:00", "During shutdown (Day 5)"),
    ("2019-11-24T12:00", "After restoration"),
]

print("\nAS58224 (TIC) prefix visibility during Nov 2019 shutdown:")
print("-"*60)

prefix_counts = []
for time_point, label in time_points:
    ris_data = get_ris_prefixes(58224, time_point)
    if ris_data:
        prefixes = ris_data.get('prefixes', {})
        v4_originated = prefixes.get('v4', {}).get('originating', [])
        v4_transiting = prefixes.get('v4', {}).get('transiting', [])
        count = len(v4_originated) + len(v4_transiting)
        prefix_counts.append({'time': time_point, 'label': label, 'prefixes': count})
        print(f"  {label}: {count} prefixes visible")
    time.sleep(1)

In [ ]:
# Analyze multiple major Iranian ASNs during shutdown
print("\n" + "="*60)
print("MULTI-ASN ANALYSIS - NOV 2019 SHUTDOWN")
print("="*60)

asns_to_check = [
    (58224, "TIC"),
    (12880, "DCI"),
    (197207, "MCI Mobile"),
    (44244, "Irancell"),
    (31549, "Shatel"),
]

shutdown_analysis = []

for asn, name in asns_to_check:
    print(f"\nAnalyzing AS{asn} ({name})...")
    
    # Before shutdown
    before = get_ris_prefixes(asn, "2019-11-14T12:00")
    before_count = 0
    if before:
        prefixes = before.get('prefixes', {})
        before_count = len(prefixes.get('v4', {}).get('originating', []))
    
    time.sleep(0.5)
    
    # During shutdown
    during = get_ris_prefixes(asn, "2019-11-18T12:00")
    during_count = 0
    if during:
        prefixes = during.get('prefixes', {})
        during_count = len(prefixes.get('v4', {}).get('originating', []))
    
    time.sleep(0.5)
    
    # After restoration
    after = get_ris_prefixes(asn, "2019-11-25T12:00")
    after_count = 0
    if after:
        prefixes = after.get('prefixes', {})
        after_count = len(prefixes.get('v4', {}).get('originating', []))
    
    change_pct = ((during_count - before_count) / before_count * 100) if before_count > 0 else 0
    
    shutdown_analysis.append({
        'ASN': f'AS{asn}',
        'Name': name,
        'Before': before_count,
        'During': during_count,
        'After': after_count,
        'Change%': f"{change_pct:.1f}%"
    })
    
    print(f"  Before: {before_count}, During: {during_count}, After: {after_count} ({change_pct:.1f}% change)")
    time.sleep(0.5)

df_shutdown = pd.DataFrame(shutdown_analysis)
print("\n" + df_shutdown.to_string(index=False))

In [ ]:
# Get BGP update stream data - actual announcements and withdrawals
print("\n" + "="*60)
print("BGP UPDATE STREAM ANALYSIS")
print("="*60)

def get_bgp_update_stream(resource, starttime, endtime):
    """Get detailed BGP update stream"""
    url = f"https://stat.ripe.net/data/bgp-update-activity/data.json?resource={resource}&starttime={starttime}&endtime={endtime}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

# Get update activity for TIC during shutdown
print("\nBGP Update Activity for AS58224 (TIC) - Nov 2019:")
update_activity = get_bgp_update_stream("AS58224", "2019-11-14", "2019-11-25")

if update_activity:
    activity_data = update_activity.get('activity', [])
    print(f"Activity data points: {len(activity_data)}")
    
    # Parse into time series
    if activity_data:
        times = []
        announcements = []
        withdrawals = []
        
        for entry in activity_data:
            timestamp = entry.get('timestamp', '')
            if timestamp:
                times.append(datetime.fromisoformat(timestamp.replace('Z', '+00:00').replace('+00:00', '')))
                announcements.append(entry.get('announcements', 0))
                withdrawals.append(entry.get('withdrawals', 0))
        
        if times:
            print(f"\nTime range: {min(times)} to {max(times)}")
            print(f"Total announcements: {sum(announcements)}")
            print(f"Total withdrawals: {sum(withdrawals)}")

In [ ]:
# Analyze September 2022 shutdown (Mahsa Amini protests)
print("\n" + "="*60)
print("SEPTEMBER 2022 SHUTDOWN ANALYSIS")
print("="*60)

# Similar analysis for Sep 2022
print("\nAnalyzing AS58224 (TIC) prefix visibility - Sep 2022:")

sep2022_times = [
    ("2022-09-19T12:00", "Before (Sep 19)"),
    ("2022-09-21T18:00", "Shutdown starts (Sep 21)"),
    ("2022-09-23T12:00", "During (Sep 23)"),
    ("2022-09-30T12:00", "Week later (Sep 30)"),
]

for time_point, label in sep2022_times:
    ris_data = get_ris_prefixes(58224, time_point)
    if ris_data:
        prefixes = ris_data.get('prefixes', {})
        v4_orig = len(prefixes.get('v4', {}).get('originating', []))
        print(f"  {label}: {v4_orig} prefixes originated")
    time.sleep(1)

In [ ]:
# Get actual routing table entries
print("\n" + "="*60)
print("ROUTING TABLE SNAPSHOT COMPARISON")
print("="*60)

def get_looking_glass(resource, timestamp):
    """Get routing table snapshot from RIS"""
    url = f"https://stat.ripe.net/data/looking-glass/data.json?resource={resource}&timestamp={timestamp}"
    try:
        resp = requests.get(url, timeout=60)
        if resp.status_code == 200:
            return resp.json().get('data', {})
    except Exception as e:
        print(f"Error: {e}")
    return {}

# Sample Iranian prefix
test_prefix = "5.160.0.0/16"  # A major Iranian prefix

print(f"\nLooking glass data for {test_prefix}:")

for time_point, label in [("2019-11-14T12:00", "Before Nov 2019"), 
                           ("2019-11-18T12:00", "During Nov 2019")]:
    lg_data = get_looking_glass(test_prefix, time_point)
    if lg_data:
        rrcs = lg_data.get('rrcs', [])
        total_peers = sum(len(rrc.get('peers', [])) for rrc in rrcs)
        print(f"  {label}: Seen by {total_peers} peers across {len(rrcs)} RRCs")
    time.sleep(1)

In [ ]:
# Export collected data
if shutdown_analysis:
    df_shutdown.to_csv('nov2019_shutdown_data.csv', index=False)
    print("Exported: nov2019_shutdown_data.csv")

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)